In [1]:
required_training = True

In [2]:
%matplotlib inline

In [3]:
import matplotlib.pyplot as plt
import os
import time

from typing import Iterable
from dataclasses import dataclass

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from torchvision import datasets, transforms
import torchvision

In [4]:
class AudioNet(nn.Module):
    def __init__(self):
        super().__init__()

        ### self._body NOT USED
        self._body = nn.Sequential(
        # First convolution layer
        # input size = (32, 32), output size = (30, 30)
        nn.Conv2d(in_channels=3, out_channels=64, kernel_size=5),
        nn.BatchNorm2d(64),
        nn.ReLU(inplace=True),
        nn.MaxPool2d(kernel_size=2),
        
        # Second convolution layer
        # input size = (15, 15), output size = (12, 12)
        nn.Conv2d(in_channels=64, out_channels=128, kernel_size=5),
        nn.BatchNorm2d(128),
        nn.ReLU(inplace=True),
        nn.MaxPool2d(kernel_size=2),
        # output size = (6, 6)
        
        # Third convolutional layer
        nn.Conv2d(in_channels=128, out_channels=256, kernel_size=5),
        nn.BatchNorm2d(256),
        nn.ReLU(inplace=True),
        
        )  
        
        self._head = nn.Sequential(
        # First fully connected layer
        nn.Linear(in_features=92, out_features=120),
        nn.ReLU(inplace=True),
            
        # Second fully connected layer
        nn.Linear(in_features=120, out_features=84),
        nn.ReLU(inplace=True),
        
        # Output Layer
        nn.Linear(in_features=84, out_features=35531)
        )
        
    def forward(self, x):
        #x = self._body(x)
        x = x.view(x.size()[0], -1)
        x = self._head(x)
        
        return x

In [5]:
audioNet_model = AudioNet()
print(audioNet_model)

AudioNet(
  (_body): Sequential(
    (0): Conv2d(3, 64, kernel_size=(5, 5), stride=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(64, 128, kernel_size=(5, 5), stride=(1, 1))
    (5): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU(inplace=True)
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Conv2d(128, 256, kernel_size=(5, 5), stride=(1, 1))
    (9): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU(inplace=True)
  )
  (_head): Sequential(
    (0): Linear(in_features=92, out_features=120, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=120, out_features=84, bias=True)
    (3): ReLU(inplace=True)
    (4): Linear(in_features=84, out_features=35531, bias=True

In [53]:
def get_data(batch_size, data_root, num_workers=4, subset = True):
    
    preprocess = transforms.Compose([  # DataLoader loads images with 3 color channels by default
    transforms.Grayscale(num_output_channels=1), # convert to one channel image 
    transforms.ToTensor() # flatten to tensor for AudioNet input
    ])
    
    train_data_path = os.path.join(data_root, 'train', 'train') # train dataloader path
    validation_data_path = os.path.join(data_root, 'validation', 'validation') # test dataloader path
    
    train_data = datasets.ImageFolder(root=train_data_path, transform=preprocess)
    validation_data = datasets.ImageFolder(root=validation_data_path, transform=preprocess)
    
    print("Training Data Length", len(train_data))
    print("Validation Data Length", len(validation_data))
    
    if subset: # use a subset of the data
        subset_size = .05 # use 5% of the data
        train_data = torch.utils.data.Subset(train_data, np.arange(0, len(train_data), int(1/subset_size)))
        validation_data = torch.utils.data.Subset(validation_data, np.arange(0, len(validation_data), int(1/subset_size)))

    train_loader = torch.utils.data.DataLoader(
        train_data,
        batch_size=batch_size,
        shuffle=True,
        num_workers=num_workers
    )

    validation_loader = torch.utils.data.DataLoader(
        validation_data,
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers
    )
    
    return train_loader, validation_loader

In [54]:
@dataclass
class SystemConfiguration:
    '''
    Describes the common system setting needed for reproducible training
    '''
    seed: int = 42  # seed number to set the state of all random number generators
    cudnn_benchmark_enabled: bool = True  # enable CuDNN benchmark for the sake of performance
    cudnn_deterministic: bool = True  # make cudnn deterministic (reproducible training)

In [55]:
@dataclass
class TrainingConfiguration:
    '''
    Describes configuration of the training process
    '''
    batch_size: int = 16  # amount of data to pass through the network at each forward-backward iteration
    epochs_count: int = 10  # number of times the whole dataset will be passed through the network
    learning_rate: float = 0.1  # determines the speed of network's weights update
        
    log_interval: int = 100  # how many batches to wait between logging training status
    test_interval: int = 1  # how many epochs to wait before another test. Set to 1 to get val loss at each epoch
    data_root: str = "/Users/gbanuru/PycharmProjects/HACKUCI/msmd/tutorials/data_root"  # folder to read/save data
    num_workers: int = 10  # number of concurrent processes using to prepare data
    device: str = 'cuda'  # device to use for training.

In [56]:
def setup_system(system_config: SystemConfiguration) -> None:
    torch.manual_seed(system_config.seed)
    if torch.cuda.is_available():
        torch.backends.cudnn_benchmark_enabled = system_config.cudnn_benchmark_enabled
        torch.backends.cudnn.deterministic = system_config.cudnn_deterministic

In [57]:
def train(
    train_config: TrainingConfiguration, model: nn.Module, optimizer: torch.optim.Optimizer,
    train_loader: torch.utils.data.DataLoader, epoch_idx: int
) -> None:
    
    # change model in training mood
    model.train()
    
    # to get batch loss
    batch_loss = np.array([])
    
    # to get batch accuracy
    batch_acc = np.array([])
        
    for batch_idx, (data, target) in enumerate(train_loader):
        
        # clone target
        indx_target = target.clone()
        # send data to device (its is medatory if GPU has to be used)
        data = data.to(train_config.device)
        # send target to device
        target = target.to(train_config.device)

        # reset parameters gradient to zero
        optimizer.zero_grad()
        
        # forward pass to the model
        output = model(data)
        
        # cross entropy loss
        loss = F.cross_entropy(output, target)
        
        # find gradients w.r.t training parameters
        loss.backward()
        # Update parameters using gardients
        optimizer.step()
        
        batch_loss = np.append(batch_loss, [loss.item()])
        
        # Score to probability using softmax
        prob = F.softmax(output, dim=1)
            
        # get the index of the max probability
        pred = prob.data.max(dim=1)[1]  
                        
        # correct prediction
        correct = pred.cpu().eq(indx_target).sum()
            
        # accuracy
        acc = float(correct) / float(len(data))
        
        batch_acc = np.append(batch_acc, [acc])

        if batch_idx % train_config.log_interval == 0 and batch_idx > 0:              
            print(
                'Train Epoch: {} [{}/{}] Loss: {:.6f} Acc: {:.4f}'.format(
                    epoch_idx, batch_idx * len(data), len(train_loader.dataset), loss.item(), acc
                )
            )
            
    epoch_loss = batch_loss.mean()
    epoch_acc = batch_acc.mean()
    return epoch_loss, epoch_acc

In [58]:
def validate(
    train_config: TrainingConfiguration,
    model: nn.Module,
    test_loader: torch.utils.data.DataLoader,
) -> float:
    model.eval()
    test_loss = 0
    count_corect_predictions = 0
    for data, target in test_loader:
        indx_target = target.clone()
        data = data.to(train_config.device)
        
        target = target.to(train_config.device)
        
        output = model(data)
        # add loss for each mini batch
        test_loss += F.cross_entropy(output, target).item()
        
        # Score to probability using softmax
        prob = F.softmax(output, dim=1)
        
        # get the index of the max probability
        pred = prob.data.max(dim=1)[1] 
        
        # add correct prediction count
        count_corect_predictions += pred.cpu().eq(indx_target).sum()

    # average over number of mini-batches
    test_loss = test_loss / len(test_loader)  
    
    # average over number of dataset
    accuracy = 100. * count_corect_predictions / len(test_loader.dataset)
    
    print(
        '\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
            test_loss, count_corect_predictions, len(test_loader.dataset), accuracy
        )
    )
    return test_loss, accuracy/100.0

In [59]:
def save_model(model, device, model_dir='models', model_file_name='audionet_model.pt'):
    

    if not os.path.exists(model_dir):
        os.makedirs(model_dir)

    model_path = os.path.join(model_dir, model_file_name)

    # make sure you transfer the model to cpu.
    if device == 'cuda':
        model.to('cpu')

    # save the state_dict
    torch.save(model.state_dict(), model_path)
    
    if device == 'cuda':
        model.to('cuda')
    
    return

In [60]:
def main(system_configuration=SystemConfiguration(), training_configuration=TrainingConfiguration()):
    
    # system configuration
    setup_system(system_configuration)

    # batch size
    batch_size_to_set = training_configuration.batch_size
    # num_workers
    num_workers_to_set = training_configuration.num_workers
    # epochs
    epoch_num_to_set = training_configuration.epochs_count

    # if GPU is available use training config, 
    # else lowers batch_size, num_workers and epochs count
    if torch.cuda.is_available():
        device = "cuda"
    else:
        device = "cpu"
        num_workers_to_set = 2

    # data loader
    train_loader, test_loader = get_data(
        batch_size=training_configuration.batch_size,
        data_root=training_configuration.data_root,
        num_workers=num_workers_to_set
    )
    
    # Update training configuration
    training_configuration = TrainingConfiguration(
        device=device,
        num_workers=num_workers_to_set
    )

    # initiate model
    model = AudioNet()
        
    # send model to device (GPU/CPU)
    model.to(training_configuration.device)

    # optimizer
    optimizer = optim.SGD(
        model.parameters(),
        lr=training_configuration.learning_rate
    )

    best_loss = torch.tensor(np.inf)
    
    # epoch train/test loss
    epoch_train_loss = np.array([])
    epoch_test_loss = np.array([])
    
    # epch train/test accuracy
    epoch_train_acc = np.array([])
    epoch_test_acc = np.array([])
    
    # trainig time measurement
    t_begin = time.time()
    for epoch in range(training_configuration.epochs_count):
        
        train_loss, train_acc = train(training_configuration, model, optimizer, train_loader, epoch)
        
        epoch_train_loss = np.append(epoch_train_loss, [train_loss])
        
        epoch_train_acc = np.append(epoch_train_acc, [train_acc])

        elapsed_time = time.time() - t_begin
        speed_epoch = elapsed_time / (epoch + 1)
        speed_batch = speed_epoch / len(train_loader)
        eta = speed_epoch * training_configuration.epochs_count - elapsed_time
        
        print(
            "Elapsed {:.2f}s, {:.2f} s/epoch, {:.2f} s/batch, ets {:.2f}s".format(
                elapsed_time, speed_epoch, speed_batch, eta
            )
        )

        if epoch % training_configuration.test_interval == 0:
            current_loss, current_accuracy = validate(training_configuration, model, test_loader)
            
            epoch_test_loss = np.append(epoch_test_loss, [current_loss])
        
            epoch_test_acc = np.append(epoch_test_acc, [current_accuracy])
            
            if current_loss < best_loss:
                best_loss = current_loss
                print('Loss decreases, saving the model.\n')
                save_model(model, device)
                
    print("Total time: {:.2f}, Best Loss: {:.3f}".format(time.time() - t_begin, best_loss))
    
    return model, epoch_train_loss, epoch_train_acc, epoch_test_loss, epoch_test_acc

In [ ]:
if required_training:
    model, epoch_train_loss, epoch_train_acc, epoch_test_loss, epoch_test_acc = main()

Training Data Length 906425
Validation Data Length 293785
Train Epoch: 0 [1600/45322] Loss: 10.441689 Acc: 0.0625
Train Epoch: 0 [3200/45322] Loss: 10.468243 Acc: 0.0000
Train Epoch: 0 [4800/45322] Loss: 10.514461 Acc: 0.0000
Train Epoch: 0 [6400/45322] Loss: 10.284564 Acc: 0.0625
Train Epoch: 0 [8000/45322] Loss: 10.458291 Acc: 0.0000
Train Epoch: 0 [9600/45322] Loss: 10.158900 Acc: 0.0625
Train Epoch: 0 [11200/45322] Loss: 10.243491 Acc: 0.0000
Train Epoch: 0 [12800/45322] Loss: 10.141673 Acc: 0.0000
Train Epoch: 0 [14400/45322] Loss: 9.611820 Acc: 0.0000
Train Epoch: 0 [16000/45322] Loss: 10.227528 Acc: 0.0000
Train Epoch: 0 [17600/45322] Loss: 8.714122 Acc: 0.0000
Train Epoch: 0 [19200/45322] Loss: 8.033529 Acc: 0.0000
Train Epoch: 0 [20800/45322] Loss: 9.375345 Acc: 0.0000
Train Epoch: 0 [22400/45322] Loss: 9.889349 Acc: 0.0000
Train Epoch: 0 [24000/45322] Loss: 8.497249 Acc: 0.0625
Train Epoch: 0 [25600/45322] Loss: 9.597708 Acc: 0.0000
Train Epoch: 0 [27200/45322] Loss: 9.347157

Train Epoch: 4 [38400/45322] Loss: 4.913227 Acc: 0.3750
Train Epoch: 4 [40000/45322] Loss: 6.462015 Acc: 0.0000
Train Epoch: 4 [41600/45322] Loss: 7.158853 Acc: 0.1250
Train Epoch: 4 [43200/45322] Loss: 5.936141 Acc: 0.0625
Train Epoch: 4 [44800/45322] Loss: 6.173256 Acc: 0.0625
Elapsed 252.49s, 50.50 s/epoch, 0.02 s/batch, ets 252.49s

Test set: Average loss: 6.2925, Accuracy: 2390/14690 (16%)

Loss decreases, saving the model.

Train Epoch: 5 [1600/45322] Loss: 6.206593 Acc: 0.0625
Train Epoch: 5 [3200/45322] Loss: 6.126093 Acc: 0.1875
Train Epoch: 5 [4800/45322] Loss: 5.796143 Acc: 0.1875
Train Epoch: 5 [6400/45322] Loss: 4.322065 Acc: 0.4375
Train Epoch: 5 [8000/45322] Loss: 6.082747 Acc: 0.1250
Train Epoch: 5 [9600/45322] Loss: 6.240104 Acc: 0.1250
Train Epoch: 5 [11200/45322] Loss: 4.402931 Acc: 0.2500
Train Epoch: 5 [12800/45322] Loss: 5.442331 Acc: 0.3750
Train Epoch: 5 [14400/45322] Loss: 6.008009 Acc: 0.1875
Train Epoch: 5 [16000/45322] Loss: 5.753677 Acc: 0.1875
Train Epoch:

In [ ]:
#Plot loss
#plt.rcParams["figure.figsize"] = (10, 6)
#x = range(len(epoch_train_loss))


#plt.figure
#plt.plot(x, epoch_train_loss, color='r', label="train loss")
#plt.plot(x, epoch_test_loss, color='b', label="validation loss")
#plt.xlabel('epoch no.')
#plt.ylabel('loss')
#plt.legend(loc='upper right')
#plt.title('Training and Validation Loss')
#plt.show()